### CSGO Match Tracker - a MongoDB project by Oliver You

### Importing necessary libraries

In [107]:
import time
import uuid
import random
import requests
from pymongo import MongoClient
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from pymongo import MongoClient
from pprint import pprint


### Web scraping and data ingestion

In [108]:
# Connecting to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["esports"]
matches_collection = db["matches"]
teams_collection = db["teams"]
players_collection = db["players"]
maps_collection = db["maps"]
player_stats_collection = db["player_stats"] 
commentary_collection = db["commentary"]

# Defining the function to fetch match data, cleaning it up, and inserting it into MongoDB
def fetch_match_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None, None, None, None, None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Extracting match title, tournament, and date
    match_title = soup.title.text.strip() if soup.title else "N/A"
    tournament_name = soup.find("p", class_="tournament-name").text.strip() if soup.find("p", class_="tournament-name") else "N/A"
    match_date = url[-10:]

    # Extracting mentions of teams from the page, then cleaning them up to get the unique team names
    team_elements = soup.find_all("div", class_="team-name")
    team_names = []
    for team_element in team_elements:
        if team_element.text.strip() in team_names:
            continue
        else:
            team_names.append(team_element.text.strip())
    team_names = team_names[::-1]

    # Extracting score from the page
    score_element = soup.find("div", class_="score")
    match_score = score_element.text.strip() if score_element else "N/A"

    # Constructing match summary data
    match_data = {
        "match_id": str(uuid.uuid4()),
        "title": match_title,
        "tournament": tournament_name,
        "date": match_date,
        "teams": team_names,
        "score": match_score,
    }

    # Extracting map names
    map_elements = soup.find_all("div", class_="map-name")
    map_names = [m.text.strip() for m in map_elements if m.text.strip() != "Full match"]

    # Finding all table rows that may contain player info
    player_rows = soup.select("div.table-row")

    # Filtering and gathering player statistics from the rows
    all_players_data = []
    all_player_stats_data = []

    for row in player_rows:
        player_name_cell = row.select_one(".table-cell.player")
        kills_cell = row.select_one(".table-cell.kills")
        deaths_cell = row.select_one(".table-cell.deaths")
        assists_cell = row.select_one(".table-cell.assists")

        # Check if row has all required cells
        if not (player_name_cell and kills_cell and deaths_cell and assists_cell):
            continue

        player_name = player_name_cell.get_text(strip=True)

        # Remove "MVP" from player name if present, since it would detect the "MVP" tag as player name
        if "MVP" in player_name:
            player_name = player_name.replace("MVP", "").strip()

        # Skip team names or "Total" rows, since they are not players
        if player_name in team_names or player_name.lower() == "total":
            continue

        # Extracting stats
        kills_p = kills_cell.select_one("p.value.underlined") or kills_cell.select_one("p.value")
        deaths_p = deaths_cell.select_one("p.value.underlined") or deaths_cell.select_one("p.value")
        assists_p = assists_cell.select_one("p.value.underlined") or assists_cell.select_one("p.value")

        # Extracting stats, defaulting to 0 if not present
        kills = kills_p.get_text(strip=True) if kills_p else "0"
        deaths = deaths_p.get_text(strip=True) if deaths_p else "0"
        assists = assists_p.get_text(strip=True) if assists_p else "0"

        # Check if stats are numeric
        if not (kills.isdigit() and deaths.isdigit() and assists.isdigit()):
            continue

        # Creating player profiles and appending them to the respective list
        temp_player_id = str(uuid.uuid4())
        player_data = {
            "player_id": temp_player_id,
            "name": player_name,
            "tournament": tournament_name,
        }
        all_players_data.append(player_data)

        # Creating player stats and appending them to the respective list
        player_stats_data = {
            "match_id": match_data["match_id"],
            "player_id": temp_player_id,  # This is temporary, will be replaced later with actual player id
            "kills": int(kills),
            "deaths": int(deaths),
            "assists": int(assists)
        }
        all_player_stats_data.append(player_stats_data)

    # Assigning players to teams
    players_data = []
    teams_data = []

    """Assigning players to teams, with 5 players per team. 
    The first 5 players are assigned to the first team, the next 5 players to the second team, etc.
    I tried many approaches, but this one seemed to work best given that finding team names in the HTML structure is rather complex, 
    whereas number of teams and team names are consistent."""

    for i, team_name in enumerate(team_names):
        team_id = str(uuid.uuid4())
        start_index = i * 5
        end_index = start_index + 5
        team_players = all_players_data[start_index:end_index]
        for player in team_players:
            player["team_id"] = team_id
            player["team_name"] = team_name
            players_data.append(player)

        team_data = {
            "team_id": team_id,
            "team_name": team_name,
            "players": [p["player_id"] for p in team_players],
            "tournament": tournament_name
        }
        teams_data.append(team_data)

    # Maps data creation
    maps_data = []
    for map_name in map_names:
        map_data = {
            "map_id": str(uuid.uuid4()),
            "name": map_name
        }
        maps_data.append(map_data)

    return match_data, teams_data, players_data, maps_data, all_player_stats_data

# Defining insert_data_to_db which inserts previous data into MongoDB
def insert_data_to_db(match_data, teams_data, players_data, maps_data, all_player_stats_data):
    if not match_data:
        print("No match data to insert.")
        return

    # Handling maps beforehand
    map_ids = []
    for m in maps_data:
        existing_map = maps_collection.find_one({"name": m["name"]})
        # If map already exists in the maps collection, update times_played, which defaults to 1. If not, insert
        if existing_map:
            maps_collection.update_one({"_id": existing_map["_id"]}, {"$inc": {"times_played": 1}})
            map_ids.append(existing_map["map_id"])
        else:
            m["times_played"] = 1
            maps_collection.insert_one(m)
            map_ids.append(m["map_id"])

    # Updating match data with map_ids
    match_data["map_ids"] = map_ids

    """Tries to insert new team name. It checks if the team already exists in the database. 
    If the team already exists, it updates the team_id with the existing team_id.
    Else, it inserts the new team into the database."""

    final_team_ids = {}
    for t in teams_data:
        existing_team = teams_collection.find_one({"team_name": t["team_name"], "tournament": t["tournament"]})
        if existing_team:
            final_team_ids[t["team_name"]] = str(existing_team["team_id"])
        else:
            teams_collection.insert_one(t)
            final_team_ids[t["team_name"]] = t["team_id"]

    # Creating final player ids, inserting players into the database
    final_player_ids = {}
    for p in players_data:
        # Fetch the team_id for the player's team
        team_id = final_team_ids.get(p["team_name"])
        if not team_id:
            print(f"Error: No valid team_id found for player {p['name']}. Skipping.")
            continue

        # Check if the player already exists in the database
        existing_player = players_collection.find_one({
            "name": p["name"],
            "tournament": p["tournament"]
        })

        if existing_player:
            # If the player's team_id is incorrect, update it with the correct team_id
            if existing_player.get("team_id") != team_id:
                players_collection.update_one(
                    {"_id": existing_player["_id"]},
                    {"$set": {"team_id": team_id}}
                )
            final_player_ids[p["player_id"]] = existing_player["player_id"]
        else:
            # Insert new player with the correct team_id if it doesn't exist
            p["team_id"] = team_id
            players_collection.insert_one(p)
            final_player_ids[p["player_id"]] = p["player_id"]

    # Replace team names in match_data["teams"] with their corresponding team_ids
    match_data["teams"] = [final_team_ids.get(team_name, team_name) for team_name in match_data["teams"]]

    # Inserting match data
    matches_collection.insert_one(match_data)
    print("Match data inserted successfully!")

    # Insert player stats
    inserted_player_stats_count = 0
    for stats in all_player_stats_data:
        if stats["player_id"] in final_player_ids:
            stats["player_id"] = final_player_ids[stats["player_id"]]
        else:
            continue
        player_stats_collection.insert_one(stats)
        inserted_player_stats_count += 1

    print(f"{len(teams_data)} team(s) data handled successfully!")
    print(f"{len(players_data)} player(s) data handled successfully!")
    print(f"{inserted_player_stats_count} player stats documents handled successfully!")
    print(f"{len(map_ids)} map(s) data handled successfully!")

urls = [
    "https://bo3.gg/matches/faze-vs-natus-vincere-22-05-2022",
    "https://bo3.gg/matches/spirit-vs-furia-19-05-2022",
    "https://bo3.gg/matches/ence-vs-natus-vincere-21-05-2022",
    "https://bo3.gg/matches/faze-vs-spirit-21-05-2022"
]
# Running the functions for each URL
if __name__ == "__main__":
    for url in urls:
        insert_data_to_db(*fetch_match_data(url))

Match data inserted successfully!
2 team(s) data handled successfully!
10 player(s) data handled successfully!
10 player stats documents handled successfully!
2 map(s) data handled successfully!
Match data inserted successfully!
2 team(s) data handled successfully!
10 player(s) data handled successfully!
10 player stats documents handled successfully!
2 map(s) data handled successfully!
Match data inserted successfully!
2 team(s) data handled successfully!
10 player(s) data handled successfully!
10 player stats documents handled successfully!
2 map(s) data handled successfully!
Match data inserted successfully!
2 team(s) data handled successfully!
10 player(s) data handled successfully!
10 player stats documents handled successfully!
2 map(s) data handled successfully!


### Hypothetical simulation of how real-time events would work

In [ ]:
# Establishing MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
db = client["esports"]
matches_collection = db["matches"]
teams_collection = db["teams"]
players_collection = db["players"]
commentary_collection = db["commentary"]

# Function to get a match by its name
def get_match_by_name(match_name):
    match = matches_collection.find_one({"title": match_name})
    return match

# Getting teams and players for a match
def get_teams_and_players(match):
    team_ids = match.get("teams", [])
    teams = list(teams_collection.find({"team_id": {"$in": team_ids}}))
    
    team_names = [team["team_name"] for team in teams]
    team_players = {}
    
    for team in teams:
        team_players[team["team_name"]] = list(players_collection.find({"team_id": team["team_id"]}))
    
    return team_names, team_players

def generate_random_event(match_id, team_names, team_players):
    #Generates a random commentary event.

    event_types = ["kill", "round_end", "clutch", "ace"]
    event = random.choice(event_types)
    
    if event == "kill":
        if len(team_players[team_names[0]]) < 2:
            return None
        killer_team = random.choice(team_names)
        killer = random.choice(team_players[killer_team])
        victim_team = random.choice([team for team in team_names if team != killer_team])
        victim = random.choice(team_players[victim_team])
        text = f"{killer['name']} killed {victim['name']}"
        tags = ["kill"]
    elif event == "round_end":
        winning_team = random.choice(team_names)
        text = f"Round end: won by {winning_team}"
        tags = ["round_end"]
    elif event == "clutch":
        team = random.choice(team_names)
        player = random.choice(team_players[team])
        text = f"{player['name']} clutched this round"
        tags = ["clutch"]
    elif event == "ace":
        team = random.choice(team_names)
        player = random.choice(team_players[team])
        text = f"{player['name']} aced this round"
        tags = ["ace"]
    else:
        return None
    
    commentary = {
        "commentary_id": str(uuid.uuid4()),
        "match_id": match_id,          
        "timestamp": datetime.now(),   
        "text": text,
        "tags": tags
    }
    
    return commentary

def run_commentary_simulation():
    print("Welcome to the Commentary Simulation!")
    print("The simulation will run for 2 minutes.")
    print("This is a hypothetical application of how it will be in a real-life match.\n")
    print("If you want to stop the simulation, press Ctrl+C.\n")
    print("Here are some examples of match names as input:")
    print("FaZe vs Natus Vincere - CS2 Match at PGL Major Antwerp 2022")
    print("Spirit vs FURIA - CS2 Match at PGL Major Antwerp 2022")
    
    match_name = input("Please enter the match name: ").strip()
    match = get_match_by_name(match_name)
    
    if not match:
        print(f"Match '{match_name}' not found in the database.")
        return
    
    team_names, players = get_teams_and_players(match)
    
    if not team_names or not players:
        print("Insufficient data to run the simulation.")
        return
    
    print(f"\nStarting commentary simulation for match '{match_name}'...")
    print("Simulation will run for 2 minutes, generating events every 15 seconds.\n")
    
    simulation_duration = 120  
    event_interval = 15         
    total_events = simulation_duration // event_interval
    start_time = datetime.now()
    match_start_time = start_time  
    
    for event_number in range(total_events):
        current_elapsed = timedelta(seconds=event_number * event_interval)
        event_timestamp = match_start_time + current_elapsed
        
        event = generate_random_event(match["match_id"], team_names, players)
        if event:
            event["timestamp"] = event_timestamp
            commentary_collection.insert_one(event)
            print(f"[{event_timestamp.strftime('%H:%M:%S')}] {event['text']} (Tags: {', '.join(event['tags'])})")
        else:
            print(f"[{event_timestamp.strftime('%H:%M:%S')}] No valid event generated.")
        
        time.sleep(event_interval)
    
    print("\nCommentary simulation ended.")

if __name__ == "__main__":
    run_commentary_simulation()

Welcome to the Commentary Simulation!
The simulation will run for 2 minutes.
This is a hypothetical application of how it will be in a real-life match.

If you want to stop the simulation, press Ctrl+C.

Here are some examples of match names as input:
FaZe vs Natus Vincere - CS2 Match at PGL Major Antwerp 2022
Spirit vs FURIA - CS2 Match at PGL Major Antwerp 2022

Starting commentary simulation for match 'Spirit vs FURIA - CS2 Match at PGL Major Antwerp 2022'...
Simulation will run for 2 minutes, generating events every 15 seconds.

[22:50:59] karrigan aced this round (Tags: ace)
[22:51:14] Round end: won by Spirit (Tags: round_end)
[22:51:29] arT clutched this round (Tags: clutch)
[22:51:44] Round end: won by Spirit (Tags: round_end)
[22:51:59] rain killed drop (Tags: kill)
[22:52:14] arT aced this round (Tags: ace)
[22:52:29] karrigan aced this round (Tags: ace)
[22:52:44] broky killed yuurih (Tags: kill)

Commentary simulation ended.


### Verification 1: Making sure that players belong to only one team

In [113]:
def verify_single_team_per_player():
    # Aggregate players by name and count distinct team_ids
    pipeline = [
        {
            "$group": {                                 
                "_id": "$name",
                "team_count": {"$addToSet": "$team_id"},
                "count": {"$sum": 1}
            }
        },
        {
            "$project": {
                "team_count": {"$size": "$team_count"},
                "count": 1
            }
        },
        {
            "$match": {
                "team_count": {"$gt": 1}
            }
        }
    ]

    inconsistent_players = list(players_collection.aggregate(pipeline))

    if inconsistent_players:
        print("Players associated with multiple teams:")
        for player in inconsistent_players:
            print(f"Player Name: {player['_id']}, Number of Teams: {player['team_count']}")
    else:
        print("All players are associated with exactly one team.")

# Run the verification
verify_single_team_per_player()

All players are associated with exactly one team.


### Verification 2: Verify that all team_ids referenced in matches exist in the teams collection

In [114]:
def verify_teams_in_matches():
    # Retrieve all team_ids from the teams collection
    existing_team_ids = set(teams_collection.distinct("team_id"))

    # Retrieve all team_ids referenced in matches
    referenced_team_ids = matches_collection.distinct("teams")

    # Find team_ids in matches that do not exist in teams collection
    invalid_team_ids = set(referenced_team_ids) - existing_team_ids

    if invalid_team_ids:
        print("The following team_ids referenced in matches do not exist in the teams collection:")
        for team_id in invalid_team_ids:
            print(team_id)
    else:
        print("All team_ids referenced in matches exist in the teams collection.")

# Run the verification
verify_teams_in_matches()

All team_ids referenced in matches exist in the teams collection.


### Verification 3: All player ids in player stats are part of the players collection

In [115]:
def verify_player_stats_references():
    # Retrieve all player_ids from the players collection
    existing_player_ids = set(players_collection.distinct("player_id"))

    # Retrieve all player_ids referenced in player_stats
    referenced_player_ids = player_stats_collection.distinct("player_id")

    # Find player_ids in player_stats that do not exist in players collection
    invalid_player_ids = set(referenced_player_ids) - existing_player_ids

    if invalid_player_ids:
        print("The following player_ids in player_stats do not exist in the players collection:")
        for player_id in invalid_player_ids:
            print(player_id)
    else:
        print("All player_ids in player_stats are valid and exist in the players collection.")

# Run the verification
verify_player_stats_references()

All player_ids in player_stats are valid and exist in the players collection.


### Verification 4: All map_ids referenced in matches are part of the maps collection

In [116]:
def verify_maps_in_matches():
    # Retrieve all map_ids from the maps collection
    existing_map_ids = set(maps_collection.distinct("map_id"))

    # Retrieve all map_ids referenced in matches
    referenced_map_ids = matches_collection.distinct("map_ids")

    # Find map_ids in matches that do not exist in maps collection
    invalid_map_ids = set(referenced_map_ids) - existing_map_ids

    if invalid_map_ids:
        print("The following map_ids referenced in matches do not exist in the maps collection:")
        for map_id in invalid_map_ids:
            print(map_id)
    else:
        print("All map_ids referenced in matches exist in the maps collection.")

# Run the verification
verify_maps_in_matches()

All map_ids referenced in matches exist in the maps collection.


### Verification 5: All commentary documents have a valid, existing match_id in the matches collection

In [117]:
def verify_commentary_matches():
    # Retrieve all match_ids from the matches collection
    existing_match_ids = set(matches_collection.distinct("match_id"))

    # Retrieve all match_ids referenced in commentary
    referenced_match_ids = commentary_collection.distinct("match_id")

    # Find match_ids in commentary that do not exist in matches collection
    invalid_match_ids = set(referenced_match_ids) - existing_match_ids

    if invalid_match_ids:
        print("The following match_ids in commentary do not exist in the matches collection:")
        for match_id in invalid_match_ids:
            print(match_id)
    else:
        print("All match_ids in commentary are valid and exist in the matches collection.")

# Run the verification
verify_commentary_matches()

All match_ids in commentary are valid and exist in the matches collection.


### Aggregate query 1: Match overviews

In [123]:
def create_match_summaries():

    #Generates comprehensive summaries for all matches in the database.
  
    pipeline = [
        # Lookup teams
        {
            "$lookup": {
                "from": "teams",
                "localField": "teams",
                "foreignField": "team_id",
                "as": "team_details"
            }
        },
        # Unwind teams array
        {
            "$unwind": "$team_details"
        },
        # Lookup players from teams
        {
            "$lookup": {
                "from": "players",
                "localField": "team_details.players",
                "foreignField": "player_id",
                "as": "team_details.players_info"
            }
        },
        # Group teams back into an array
        {
            "$group": {
                "_id": "$match_id",
                "title": { "$first": "$title" },
                "tournament": { "$first": "$tournament" },
                "date": { "$first": "$date" },
                "score": { "$first": "$score" },
                "map_ids": { "$first": "$map_ids" },
                "team_details": { "$push": "$team_details" }
            }
        },
        # Lookup map names
        {
            "$lookup": {
                "from": "maps",
                "localField": "map_ids",
                "foreignField": "map_id",
                "as": "map_details"
            }
        },
        # Lookup key commentary events
        {
            "$lookup": {
                "from": "commentary",
                "let": { "matchId": "$_id" },
                "pipeline": [
                    {
                        "$match": {
                            "$expr": { "$eq": ["$match_id", "$$matchId"] }
                        }
                    },
                    {
                        "$sort": { "timestamp": -1 }
                    },
                    {
                        "$limit": 3
                    }
                ],
                "as": "key_events"
            }
        },
        # Final projection
        {
            "$project": {
                "_id": 0,
                "match_id": "$_id",
                "title": 1,
                "tournament": 1,
                "date": 1,
                "score": 1,
                "maps_played": "$map_details.name",
                "teams": {
                    "$map": {
                        "input": "$team_details",
                        "as": "team",
                        "in": {
                            "team_id": "$$team.team_id",
                            "team_name": "$$team.team_name",
                            "players": {
                                "$map": {
                                    "input": "$$team.players_info",
                                    "as": "player",
                                    "in": {
                                        "player_id": "$$player.player_id",
                                        "name": "$$player.name"
                                    }
                                }
                            }
                        }
                    }
                },
                "key_events": {
                    "$map": {
                        "input": "$key_events",
                        "as": "event",
                        "in": {
                            "commentary_id": "$$event.commentary_id",
                            "timestamp": "$$event.timestamp",
                            "text": "$$event.text",
                            "tags": "$$event.tags"
                        }
                    }
                }
            }
        },
        # Sort matches by date in ascending order
        {
            "$sort": { "date": 1 }
        }
    ]
    
    # Execute the aggregation pipeline
    summaries = list(matches_collection.aggregate(pipeline))
    
    if not summaries:
        print("No matches found in the database.")
        return
    
    #Print each match summary
    for summary in summaries:
        print("="*80)
        print(f"Match ID      : {summary['match_id']}")
        print(f"Title         : {summary['title']}")
        print(f"Tournament    : {summary['tournament']}")
        print(f"Date          : {summary['date']}")
        print(f"Score         : {summary['score']}")
        print(f"Maps Played   : {', '.join(summary['maps_played'])}")
        print("\nTeams:")
        for team in summary['teams']:
            print(f"  - Team ID   : {team['team_id']}")
            print(f"    Team Name : {team['team_name']}")
            print(f"    Players   :")
            for player in team['players']:
                print(f"      * {player['name']} (ID: {player['player_id']})")
            print()
        print("Key Events:")
        for event in summary['key_events']:
            event_time = event['timestamp'].strftime("%Y-%m-%d %H:%M:%S")
            print(f"  [{event_time}] {event['text']} (Tags: {', '.join(event['tags'])})")
        print("="*80 + "\n")

# Example Usage
if __name__ == "__main__":
    create_match_summaries()

Match ID      : aba98f4a-b100-417c-a06d-c80b0adf0d6a
Title         : Spirit vs FURIA - CS2 Match at PGL Major Antwerp 2022
Tournament    : PGL Major Antwerp 2022
Date          : 19-05-2022
Score         : 2 - 0
Maps Played   : Vertigo, Ancient

Teams:
  - Team ID   : da654954-a9ea-4a32-83ef-0dca920513a1
    Team Name : Spirit
    Players   :
      * s1ren (ID: f5b7a14e-e314-4d42-828e-ffbed0ef7777)
      * degster (ID: 8a4c5ec2-0576-4873-96af-98eb4723a545)
      * chopper (ID: d84eeef6-c1ef-4617-9f29-08853b03f1f9)
      * magixx (ID: 01f7c374-7dcf-49a6-8611-fd5dc6ffa49b)
      * Patsi (ID: 06ab084f-c6b5-4f96-8dbd-80b973c4ea4f)

  - Team ID   : a6fc7833-5771-4992-a5ea-4f7adc9c9a74
    Team Name : FURIA
    Players   :
      * yuurih (ID: f3133574-d021-4df1-80fb-5c5918210014)
      * saffee (ID: ebc5965a-2046-4d0f-a29f-de3ca07b0109)
      * arT (ID: 41a9d380-df46-4250-94b2-3435ccd8ac21)
      * drop (ID: caf73a3a-2028-4132-9037-03914bdaddcd)
      * KSCERATO (ID: b84bf58e-c152-4a46-bad1-9

### Aggregate 2: Top 10 players with the highest KDA

In [ ]:
def top_players_best_kda(top_n=10):

    pipeline = [
        # Group player statistics by player_id
        {
            "$group": {
                "_id": "$player_id",
                "total_kills": {"$sum": "$kills"},
                "total_deaths": {"$sum": "$deaths"},
                "total_assists": {"$sum": "$assists"}
            }
        },
        # Calculate KDA, handling division by zero
        {
            "$addFields": {
                "kda": {
                    "$cond": [
                        {"$eq": ["$total_deaths", 0]},
                        {"$add": ["$total_kills", "$total_assists"]},
                        {
                            "$divide": [
                                {"$add": ["$total_kills", "$total_assists"]},
                                "$total_deaths"
                            ]
                        }
                    ]
                }
            }
        },
        # Sort players by KDA in descending order
        {
            "$sort": {"kda": -1}
        },
        # Limit to top N players
        {
            "$limit": top_n
        },
        # Lookup player information from the players collection
        {
            "$lookup": {
                "from": "players",
                "localField": "_id",
                "foreignField": "player_id",
                "as": "player_info"
            }
        },
        # Unwind the player_info array
        {
            "$unwind": "$player_info"
        },
        # Project the desired fields
        {
            "$project": {
                "_id": 0,
                "player_id": "$_id",
                "player_name": "$player_info.name",
                "total_kills": 1,
                "total_deaths": 1,
                "total_assists": 1,
                "kda": {"$round": ["$kda", 2]} 
            }
        }
    ]
    
    try:
        results = list(player_stats_collection.aggregate(pipeline))
    except Exception as e:
        print(f"An error occurred during aggregation: {e}")
        return
    
    if not results:
        print("No player statistics found in the database.")
        return
    
    # Display the top players with the best KDA
    print(f"Top {top_n} Players with the Best KDA:")
    for idx, player in enumerate(results, start=1):
        print(f"{idx}. {player['player_name']} (KDA: {player['kda']}) - Kills: {player['total_kills']}, Deaths: {player['total_deaths']}, Assists: {player['total_assists']}")

if __name__ == "__main__":
    top_players_best_kda(top_n=10)

Top 10 Players with the Best KDA:
1. rain (KDA: 1.58) - Kills: 117, Deaths: 93, Assists: 30
2. broky (KDA: 1.55) - Kills: 105, Deaths: 80, Assists: 19
3. degster (KDA: 1.47) - Kills: 88, Deaths: 70, Assists: 15
4. Perfecto (KDA: 1.47) - Kills: 80, Deaths: 66, Assists: 17
5. s1mple (KDA: 1.46) - Kills: 103, Deaths: 83, Assists: 18
6. chopper (KDA: 1.43) - Kills: 90, Deaths: 82, Assists: 27
7. b1t (KDA: 1.38) - Kills: 69, Deaths: 64, Assists: 19
8. ropz (KDA: 1.35) - Kills: 96, Deaths: 84, Assists: 17
9. Twistzz (KDA: 1.29) - Kills: 85, Deaths: 86, Assists: 26
10. s1ren (KDA: 1.29) - Kills: 90, Deaths: 87, Assists: 22


### Aggregate 3: Top performer each team per match

In [120]:
def top_performer_each_team_per_match():
    pipeline = [
        {
            "$lookup": {
                "from": "player_stats",
                "localField": "match_id",
                "foreignField": "match_id",
                "as": "match_player_stats"
            }
        },
        {
            "$unwind": "$match_player_stats"
        },
        {
            "$lookup": {
                "from": "players",
                "localField": "match_player_stats.player_id",
                "foreignField": "player_id",
                "as": "player_info"
            }
        },
        {
            "$unwind": "$player_info"
        },
        {
            "$addFields": {
                "kda": {
                    "$cond": [
                        {"$eq": ["$match_player_stats.deaths", 0]},
                        {"$add": ["$match_player_stats.kills", "$match_player_stats.assists"]},
                        {
                            "$divide": [
                                {"$add": ["$match_player_stats.kills", "$match_player_stats.assists"]},
                                "$match_player_stats.deaths"
                            ]
                        }
                    ]
                },
                "team_id": "$player_info.team_id",
                "team_name": "$player_info.team_name"
            }
        },
        {
            "$group": {
                "_id": {
                    "match_id": "$match_id",
                    "team_id": "$team_id",
                    "team_name": "$team_name"
                },
                "players": {
                    "$push": {
                        "player_id": "$player_info.player_id",
                        "player_name": "$player_info.name",
                        "kda": "$kda"
                    }
                }
            }
        },
        {
            "$project": {
                "_id": 0,
                "match_id": "$_id.match_id",
                "team_id": "$_id.team_id",
                "team_name": "$_id.team_name",
                "players": 1,
                "top_performer": {
                    "$arrayElemAt": [
                        {
                            "$slice": [
                                {
                                    "$filter": {
                                        "input": "$players",
                                        "as": "player",
                                        "cond": {}
                                    }
                                },
                                0,
                                1
                            ]
                        },
                        0
                    ]
                }
            }
        },
        {
            "$lookup": {
                "from": "matches",
                "localField": "match_id",
                "foreignField": "match_id",
                "as": "match_info"
            }
        },
        {
            "$unwind": "$match_info"
        },
        {
            "$project": {
                "match_id": 1,
                "match_title": "$match_info.title",
                "team_id": 1,
                "team_name": 1,
                "top_performer": 1
            }
        },
        {
            "$sort": {
                "match_id": 1,
                "team_name": 1
            }
        }
    ]
    pipeline[6]["$project"]["top_performer"] = {
        "$arrayElemAt": [
            {
                "$slice": [
                    {
                        "$filter": {
                            "input": {
                                "$sortArray": {
                                    "input": "$players",
                                    "sortBy": {"kda": -1}
                                }
                            },
                            "as": "player",
                            "cond": {}
                        }
                    },
                    0,
                    1
                ]
            },
            0
        ]
    }

    results = list(matches_collection.aggregate(pipeline))
    print("Top Performer from Each Team in Every Match:")
    for entry in results:
        print(f"Match: {entry['match_title']} | Team: {entry['team_name']}")
        print(f"  Top Performer: {entry['top_performer']['player_name']} (KDA: {round(entry['top_performer']['kda'], 2)})\n")

top_performer_each_team_per_match()

Top Performer from Each Team in Every Match:
Match: FaZe vs Natus Vincere - CS2 Match at PGL Major Antwerp 2022 | Team: FaZe
  Top Performer: rain (KDA: 1.85)

Match: FaZe vs Natus Vincere - CS2 Match at PGL Major Antwerp 2022 | Team: Natus Vincere
  Top Performer: Perfecto (KDA: 1.4)

Match: Spirit vs FURIA - CS2 Match at PGL Major Antwerp 2022 | Team: FURIA
  Top Performer: saffee (KDA: 1.16)

Match: Spirit vs FURIA - CS2 Match at PGL Major Antwerp 2022 | Team: Spirit
  Top Performer: degster (KDA: 1.87)

Match: ENCE vs Natus Vincere - CS2 Match at PGL Major Antwerp 2022 | Team: ENCE
  Top Performer: Maden (KDA: 1.0)

Match: ENCE vs Natus Vincere - CS2 Match at PGL Major Antwerp 2022 | Team: Natus Vincere
  Top Performer: s1mple (KDA: 2.23)

Match: FaZe vs Spirit - CS2 Match at PGL Major Antwerp 2022 | Team: FaZe
  Top Performer: broky (KDA: 1.42)

Match: FaZe vs Spirit - CS2 Match at PGL Major Antwerp 2022 | Team: Spirit
  Top Performer: chopper (KDA: 1.32)

